In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
import random
import numpy as np

from models import EmotionClassificationModel
from utils import prepare_data, test, freeze_all, freeze_except_last_two

In [4]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashkens. Use `wandb login --relogin` to force relogin


True

### Sweep Config

In [5]:
sweep_config = {'method': 'grid'}

metric = {
    'name': 'val F1',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [6]:
# variable parameters
parameters_dict = {
    'freezing': {
        'values': ['none', 'full', 'partial']
    },
    'seed': {
        'values': [42, 11, 100]
    }
}

sweep_config['parameters'] = parameters_dict

In [7]:
# constant parameters
parameters_dict.update({
    'epochs': {
        'value': 10
    },
    'lr': {
        'value': 1e-05
    },
    'batch_size': {
        'value': 32
    },
    'data': {
        'value': 'MeldCSV/'
    }
})

In [8]:
sweep_id = wandb.sweep(sweep_config, project="sweep-with-testing")

Create sweep with ID: qbvq1pj4
Sweep URL: https://wandb.ai/yashkens/sweep-with-testing/sweeps/qbvq1pj4


### Train

In [17]:
def train_net(config=None):
    with wandb.init(config=config) as run:
        
        config = wandb.config
        torch.manual_seed(config.seed)
        random.seed(config.seed)
        np.random.seed(config.seed)
        
        name_str = f"{config.freezing}_seed-{config.seed}"
        run.name = name_str
        
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        
        data_path = '../../' + config.data
        num_labels, train_dataloader, test_dataloader, dev_dataloader, test_dataset = prepare_data(tokenizer, 
                                                                                                   data_path, 
                                                                                                   config.batch_size,
                                                                                                   max_len=128)
        
        model = BertForSequenceClassification.from_pretrained(
            'bert-base-uncased',
            num_labels = num_labels
        )
        
        if config.freezing == 'full':
            model = freeze_all(model)
        elif config.freezing == 'partial':
            model = freeze_except_last_two(model)
        
        optimizer = torch.optim.Adam(params=model.parameters(), lr=config.lr)
        device = 'cuda'
        answer_model = EmotionClassificationModel(model, device=device)
        
        patience = 3
        answer_model.train(train_dataloader, dev_dataloader, config.epochs, optimizer, patience)
        
        test_fscore = test(test_dataset, answer_model)
        wandb.log({"test F1": test_fscore})

In [18]:
wandb.agent(sweep_id, train_net)

wandb: Agent Starting Run: 4vfrlxxw with config:
wandb: 	batch_size: 32
wandb: 	data: MeldCSV/
wandb: 	epochs: 10
wandb: 	freezing: partial
wandb: 	lr: 1e-05
wandb: 	seed: 42


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at